In [1]:
from agent import TelosAgent
from environment import TelosTaskEnv, make_env
from task import TelosTask
from standing_task.standing_task import StandingTelosTask
from standing_task.standing_environment import StandingTelosTaskEnv,make_standing_env
from time import sleep
import pybullet as p
from utils.PyBullet import PyBullet
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3 import SAC, PPO
import numpy as np
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

pybullet build time: Nov 28 2023 23:45:17


In [2]:
pb = PyBullet(render_mode="rgb_array",renderer="OpenGL")
telos_agent = TelosAgent(pb)
telos_task = StandingTelosTask(agent=telos_agent,sim_engine=pb)
telos_env = StandingTelosTaskEnv(task=telos_task, agent=telos_agent, sim_engine=pb)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (Intel(R) Iris(R) Xe Graphics)
GL_VERSION=4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.10
pthread_getconcurrency()=0
Version = 4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (Intel(R) Iris(R) Xe Graphics)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Microsoft Corporation
ven = Microsoft Corporation


numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished


In [3]:
env = make_vec_env(make_standing_env, n_envs=6, env_kwargs={"task": telos_task, "agent": telos_agent, "sim": pb})
env = VecNormalize(env, norm_obs=True, norm_reward=True)

In [9]:
env.step(env.action_space.sample())

error: number of target positions should match the number of joint indices

In [ ]:
policy_sac =  dict(net_arch=dict(pi=[512, 512,512], qf=[512, 512,512]))

In [ ]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.05 * np.ones(n_actions))

In [ ]:
model_sac = SAC(
    policy="MultiInputPolicy",
    env=env,
    policy_kwargs=policy_sac,
    action_noise= action_noise,
    learning_rate=1e-3,
    buffer_size= int(1e6),
    batch_size= 1024,
    ent_coef='auto',
    gamma= 0.995,
    tau= 0.005,
    train_freq= 1,
    gradient_steps= 10,
    optimize_memory_usage= False,
    target_update_interval= 1,
    verbose= 1,
    tensorboard_log="./tensorboard/standing_task/sacTelos/",
)

In [ ]:
model_sac.learn(5_000_000)

In [ ]:
model_sac.save("sac_standing_task_3")

In [ ]:
pb_t = PyBullet(render_mode="rgb_array",renderer="OpenGL")
telos_agent_t = TelosAgent(pb_t)
telos_task_t = StandingTelosTask(agent=telos_agent_t,sim_engine=pb_t)
telos_env_t = StandingTelosTaskEnv(task=telos_task_t, agent=telos_agent_t, sim_engine=pb_t)

In [ ]:
test_model = SAC.load("sac_standing_task")